<a href="https://colab.research.google.com/github/mammadmaheri7/M-M-1-K/blob/main/hw1_performance_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment

In [ ]:
import numpy as np
from queue import Queue
import random


class Customer:

    def __init__(self, id, arrival_time, waiting_time, service_time):
        self.arrival_time = arrival_time
        self.waiting_time = waiting_time
        self.service_time = service_time
        self.id = id

    def __str__(self):
        return "id:" + str(self.id) + "\t arrival time:" + str(self.arrival_time) + "\t waiting time:" + str(
            self.waiting_time)

    def __lt__(self, other):
        return self.arrival_time < other.arrival_time


class Event:

    def __init__(self, customer_id, type_idx, occ_time):
        self.types = ['arrival', 'deadline', 'done']
        self.type_idx = type_idx
        self.occ_time = occ_time
        self.customer_id = customer_id

    def __str__(self):
        return "customer id:" + str(self.customer_id) + "\t occurence time:" + str(self.occ_time) + "\t type:" + str(
            self.types[self.type_idx])

    def __lt__(self, other):
        return self.occ_time < other.occ_time

    def __le__(self, other):
        return self.occ_time <= other.occ_time

    def __sub__(self, other):
        if isinstance(other, Event):
            return self.occ_time - other.occ_time
        else:
            return self.occ_time - other


def generate_exp_rv(rate):
    return -1.0 * np.log(1.0 - random.random()) / rate


def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0

    while low <= high:
        mid = (high + low) // 2

        # If x is greater, ignore left half
        if arr[mid] < x:
            low = mid + 1

        # If x is smaller, ignore right half
        elif arr[mid] > x:
            high = mid - 1

        # means x is present at mid
        else:
            return mid

    # If we reach here, then the element was not present
    return low


def insert_arr(arr, value):
    # idx = np.argmin(np.abs(value - np.array(arr)))
    idx = binary_search(arr, value)
    if idx >= len(arr):
        idx = len(arr)
        arr.insert(idx, value)
    else:
        if value <= arr[idx]:
            arr.insert(idx, value)
        else:
            while idx <= len(arr) - 1 and value > arr[idx]:
                idx += 1
            arr.insert(idx, value)
    return arr

In [ ]:
# rate of arrival
lamb = 10

# rate of service time
mu = 1

# waiting time parameter
theta = 2
const_theta = True

num_of_customers = 1_000_000_0

In [ ]:
import gc
gc.collect()

234

In [ ]:
def make_customer(pre_cus_time,id):
  if const_theta:
      waiting_time = theta
  else:
      waiting_time = generate_exp_rv(1.0 / theta)

  service_time = generate_exp_rv(mu)
  arrival_time = pre_cus_time + generate_exp_rv(lamb)
  customer = Customer(id, arrival_time, waiting_time, service_time)
  return customer

In [ ]:
if __name__ == "__main__":
    # rate of arrival
    # lamb = 10

    # # rate of service time
    # mu = 1

    # # waiting time parameter
    # theta = 2
    # const_theta = True

    # num_of_customers = 100_000_000

    # # Initializing a queue
    # # q = Queue(maxsize = num_of_customers)
    # q = []

    arrival_time = 0

    # for i in range(num_of_customers):
    #     if const_theta:
    #         waiting_time = theta
    #     else:
    #         waiting_time = generate_exp_rv(1.0 / theta)

    #     service_time = generate_exp_rv(mu)
    #     customer = Customer(i, arrival_time, waiting_time, service_time)
    #     q.append(customer)
    #     arrival_time += generate_exp_rv(lamb)
    
    pre_cus_time = 0
    cus = make_customer(pre_cus_time,id=0)

    queue = Queue(maxsize=12)

    num_blocked_users = 0
    num_deadlined_users = 0

    events = []


    # for i in range(num_of_customers):
    #     events.append(Event(i, type_idx=0, occ_time=q[i].arrival_time))

    # for event in events:
    #     print(event)
    #
    # for c in q:
    #     print(c)

    def print_list(t):
        for item in t:
            print(item)


    time = 0
    cnt = 0

    events.append(Event(cnt, type_idx=0, occ_time=cus.arrival_time))
    pre_cus = cus
    cnt += 1

    nxt_cus = make_customer(cus.arrival_time,1)

    # main loop
    while len(events) > 0 or cnt<num_of_customers:
        # events.append(Event(cnt, type_idx=0, occ_time=q[cnt].arrival_time))
        if cnt < num_of_customers :
            # insert_arr(events, Event(cnt, type_idx=0, occ_time=q[cnt].arrival_time))
            if len(events)==0:
              events.append(Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
              cnt += 1
              pre_cus = nxt_cus
              nxt_cus = make_customer(pre_cus.arrival_time , cnt)
            elif nxt_cus.arrival_time < events[0].occ_time:
              events.insert(0,Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
              cnt += 1
              pre_cus = nxt_cus
              nxt_cus = make_customer(pre_cus.arrival_time , cnt)


        event = events.pop(0)
        time = event.occ_time

        # arrival customer event
        if event.type_idx == 0:
            if queue.qsize() >= 12:
                num_blocked_users += 1
            else:
                # customer = q[event.customer_id]
                customer = pre_cus
                queue.put(customer)

                if queue.qsize() == 1:
                    # events.append(Event(event.customer_id, 2, time + customer.service_time))
                    insert_arr(events, Event(event.customer_id, 2, time + customer.service_time))
                else:
                    # events.append(Event(event.customer_id, 1, time + customer.waiting_time))
                    insert_arr(events, Event(event.customer_id, 1, time + customer.waiting_time))
                # events.sort()

        # dead line event
        if event.type_idx == 1:
            # customer = q[event.customer_id]
            customer = next((x for x in queue.queue if x.id == event.customer_id), None)
            queue.get(customer)
            num_deadlined_users += 1

        # done event
        if event.type_idx == 2:
            # customer = q[event.customer_id]
            customer = next((x for x in queue.queue if x.id == event.customer_id), None)
            queue.get(customer)

            # if exist a customer in queue
            if queue.qsize() >= 1:
                cus = queue.queue[0]

                # remove deadline event
                for event in events:
                    if event.customer_id == cus.id and event.type_idx == 1:
                        events.remove(event)
                        break

                # add done event
                insert_arr(events, Event(cus.id, 2, time + cus.service_time))


    pb = num_blocked_users / num_of_customers
    pd = num_deadlined_users / num_of_customers

    print("PB:" + str(pb) + "\t\t PD:" + str(pd))


PB:0.48530239		 PD:0.41467862


# Theorical

In [ ]:
import math

theta = 2
mu = 1
lamba = 10
constant_theta = False

def phi_n (n,is_const_theta,mu,theta):
  if not is_const_theta:
    denom = 1
    for i in range(n+1):
      denom *= (mu + (i/theta))
    return math.factorial(n) / (denom)
  else:
    summation = 0
    for i in range(n):
      summation += ((mu*theta)**i / math.factorial(i))
    return ( math.factorial(n) / mu**(n+1) ) * ( 1 - math.exp(-1*mu*theta)*summation)


def p_n (n,p0,lamba,is_const_theta,mu,theta):
  if n==1:
    return p0 * lamba / mu
  else:
    return p0 * lamba**n * phi_n(n-1,is_const_theta,mu,theta) / math.factorial(n-1)

In [ ]:
def caculate_p0 (lamba,constant_theta,mu,theta):
  sum = 0
  for i in range(2,13):
    sum += lamba**i * phi_n(i-1,constant_theta,mu,theta) / math.factorial(i-1)
  p0 = 1.0 / (1 + lamba/mu + sum)
  return p0

In [ ]:
pb = p_n(12,p0,lamba,constant_theta,mu,theta)

In [ ]:
pd = 1 - pb - (mu/lamba)*(1-p0)

In [ ]:
pd

0.48994991453292386

# Excel

## Analytics - constat theta

In [ ]:
cns = True
for lamba in np.arange(0.1,20.1,0.1):
    p0 = caculate_p0(lamba,cns,mu,theta)
    pb = p_n(12,p0,lamba,cns,mu,theta)
    pd = 1 - pb - (mu/lamba)*(1-p0)
    print(pb,"\t\t",pd)

7.489782458867244e-18 		 0.0149015320066761
2.7446040007410814e-14 		 0.032566444952698004
3.1608835320832516e-12 		 0.05296075903988229
8.786776693586714e-11 		 0.07594655692204522
1.1169095614181157e-09 		 0.10128502937970396
8.630064005551852e-09 		 0.1286490055630658
4.718888170815029e-08 		 0.15764355387356566
1.9997957171314385e-07 		 0.1878318875203805
6.966426810799309e-07 		 0.2187630680070557
2.077056925921142e-06 		 0.24999802329823828
5.4561504866732605e-06 		 0.2811310980956514
1.290287656331491e-05 		 0.3118054641051724
2.7926653036665552e-05 		 0.3417219103809692
5.604104473733082e-05 		 0.3706415351794886
0.00010535276236165188 		 0.3983835197770652
0.00018710914612781208 		 0.424819456139798
0.000316132352112977 		 0.44986568858679643
0.0005110743060882779 		 0.4734749192238641
0.000794441877654494 		 0.49562802196223477
0.0011923639306305428 		 0.5163266908238417
0.00173409746668049 		 0.5355872651582703
0.002451295469490384 		 0.5534358509265925
0.003377081177991318 

## Analytics - exponential theta

In [ ]:
cns = False
for lamba in np.arange(0.1,20.1,0.1):
    p0 = caculate_p0(lamba,cns,mu,theta)
    pb = p_n(12,p0,lamba,cns,mu,theta)
    pd = 1 - pb - (mu/lamba)*(1-p0)
    print(pb,"\t\t",pd)

5.941917264466558e-19 		 0.033311132253988296
2.1912364181462115e-15 		 0.06648956343947099
2.5512402543602246e-13 		 0.09940509698818079
7.203800794836566e-12 		 0.13193244182569408
9.345989447690423e-11 		 0.16395341365351568
7.405698215917785e-10 		 0.19535885403887954
4.172001990641107e-09 		 0.22605020336022674
1.8295702196178467e-08 		 0.2559406849311857
6.622183254945918e-08 		 0.2849560795248205
2.0590786734838644e-07 		 0.31303509020103004
5.659379376554201e-07 		 0.3401293154669647
1.4043022032714312e-06 		 0.36620286361921495
3.1968716918247604e-06 		 0.39123165240027524
6.760780415886256e-06 		 0.4152024460091619
1.3415076932000509e-05 		 0.43811168641024134
2.5176997717335677e-05 		 0.45996417821908897
4.4987480490920185e-05 		 0.48077168659713365
7.695466954246381e-05 		 0.5005515058488366
0.00012659981947492466 		 0.5193250529802551
0.00020108678916375396 		 0.5371165354768878
0.0003094147157255801 		 0.5539517361090927
0.00046255375891132255 		 0.5698569498280154
0.0006

## Experiment - const theta

In [ ]:
# rate of service time
mu = 1

# waiting time parameter
theta = 2

const_theta = True
num_of_customers = 1_000_000_0

for lamba in np.arange(0.1,20.1,0.1):
    lamb = lamba
    arrival_time = 0
    pre_cus_time = 0
    cus = make_customer(pre_cus_time,id=0)

    queue = Queue(maxsize=12)

    num_blocked_users = 0
    num_deadlined_users = 0

    events = []

    time = 0
    cnt = 0

    events.append(Event(cnt, type_idx=0, occ_time=cus.arrival_time))
    pre_cus = cus
    cnt += 1

    nxt_cus = make_customer(cus.arrival_time,1)

    # main loop
    while len(events) > 0 or cnt<num_of_customers:
        if cnt < num_of_customers :
            if len(events)==0:
              events.append(Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
              cnt += 1
              pre_cus = nxt_cus
              nxt_cus = make_customer(pre_cus.arrival_time , cnt)
            elif nxt_cus.arrival_time < events[0].occ_time:
              events.insert(0,Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
              cnt += 1
              pre_cus = nxt_cus
              nxt_cus = make_customer(pre_cus.arrival_time , cnt)

        event = events.pop(0)
        time = event.occ_time

        # arrival customer event
        if event.type_idx == 0:
            if queue.qsize() >= 12:
                num_blocked_users += 1
            else:
                # customer = q[event.customer_id]
                customer = pre_cus
                queue.put(customer)

                if queue.qsize() == 1:
                    insert_arr(events, Event(event.customer_id, 2, time + customer.service_time))
                else:
                    insert_arr(events, Event(event.customer_id, 1, time + customer.waiting_time))


        # dead line event
        if event.type_idx == 1:
            customer = next((x for x in queue.queue if x.id == event.customer_id), None)
            queue.get(customer)
            num_deadlined_users += 1

        # done event
        if event.type_idx == 2:
            customer = next((x for x in queue.queue if x.id == event.customer_id), None)
            queue.get(customer)

            # if exist a customer in queue
            if queue.qsize() >= 1:
                cus = queue.queue[0]

                # remove deadline event
                for event in events:
                    if event.customer_id == cus.id and event.type_idx == 1:
                        events.remove(event)
                        break

                # add done event
                insert_arr(events, Event(cus.id, 2, time + cus.service_time))


    pb = num_blocked_users / num_of_customers
    pd = num_deadlined_users / num_of_customers

    print(pb,"\t",pd)
    gc.collect()

0.0 	 0.0149619
0.0 	 0.0324951
0.0 	 0.0531195
0.0 	 0.0760685
0.0 	 0.1011687
0.0 	 0.1289268
0.0 	 0.1577518
0.0 	 0.1883633
5e-07 	 0.2186271
2e-06 	 0.2499604
4e-06 	 0.2814623
1.42e-05 	 0.3117096
2.75e-05 	 0.3418705
5.93e-05 	 0.3712704
0.000104 	 0.3980171
0.0001908 	 0.4253036
0.0003312 	 0.4503853
0.0004967 	 0.4735724
0.0008068 	 0.4956454
0.0011772 	 0.5164763
0.0017249 	 0.5358285
0.0024745 	 0.5534913
0.0033958 	 0.5699455
0.0045789 	 0.5850965
0.0060118 	 0.5985329
0.0077802 	 0.61149
0.0097509 	 0.6226937
0.0122254 	 0.6326199
0.0150335 	 0.6417051
0.0181649 	 0.6497058
0.0219606 	 0.6566591
0.0258853 	 0.6625114
0.0302759 	 0.6672556
0.0352646 	 0.67126
0.0403212 	 0.6743762
0.0459867 	 0.6764636
0.0519903 	 0.6780366
0.0583059 	 0.6785336
0.0649141 	 0.6787825
0.0716987 	 0.6784941
0.0788725 	 0.6773207
0.0863372 	 0.6756039
0.0940055 	 0.6735651
0.1017501 	 0.6707075
0.1096992 	 0.6682225
0.1179343 	 0.6648004
0.1258938 	 0.6613568
0.1346445 	 0.6568352
0.1428636 	 

In [ ]:
# rate of service time
mu = 1

# waiting time parameter
theta = 2

const_theta = True
num_of_customers = 1_000_000_0

for lamba in np.arange(8.4,20.1,0.1):
    lamb = lamba
    arrival_time = 0
    pre_cus_time = 0
    cus = make_customer(pre_cus_time,id=0)

    queue = Queue(maxsize=12)

    num_blocked_users = 0
    num_deadlined_users = 0

    events = []

    time = 0
    cnt = 0

    events.append(Event(cnt, type_idx=0, occ_time=cus.arrival_time))
    pre_cus = cus
    cnt += 1

    nxt_cus = make_customer(cus.arrival_time,1)

    # main loop
    while len(events) > 0 or cnt<num_of_customers:
        if cnt < num_of_customers :
            if len(events)==0:
              events.append(Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
              cnt += 1
              pre_cus = nxt_cus
              nxt_cus = make_customer(pre_cus.arrival_time , cnt)
            elif nxt_cus.arrival_time < events[0].occ_time:
              events.insert(0,Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
              cnt += 1
              pre_cus = nxt_cus
              nxt_cus = make_customer(pre_cus.arrival_time , cnt)

        event = events.pop(0)
        time = event.occ_time

        # arrival customer event
        if event.type_idx == 0:
            if queue.qsize() >= 12:
                num_blocked_users += 1
            else:
                # customer = q[event.customer_id]
                customer = pre_cus
                queue.put(customer)

                if queue.qsize() == 1:
                    insert_arr(events, Event(event.customer_id, 2, time + customer.service_time))
                else:
                    insert_arr(events, Event(event.customer_id, 1, time + customer.waiting_time))


        # dead line event
        if event.type_idx == 1:
            customer = next((x for x in queue.queue if x.id == event.customer_id), None)
            queue.get(customer)
            num_deadlined_users += 1

        # done event
        if event.type_idx == 2:
            customer = next((x for x in queue.queue if x.id == event.customer_id), None)
            queue.get(customer)

            # if exist a customer in queue
            if queue.qsize() >= 1:
                cus = queue.queue[0]

                # remove deadline event
                for event in events:
                    if event.customer_id == cus.id and event.type_idx == 1:
                        events.remove(event)
                        break

                # add done event
                insert_arr(events, Event(cus.id, 2, time + cus.service_time))


    pb = num_blocked_users / num_of_customers
    pd = num_deadlined_users / num_of_customers

    print(pb,"\t",pd)
    gc.collect()

0.403748 	 0.4770955
0.4094289 	 0.4728382
0.4149385 	 0.4687959
0.4205684 	 0.4644687
0.4263354 	 0.460163
0.43178 	 0.4558271
0.4367627 	 0.4522574
0.4417728 	 0.4484533
0.4473182 	 0.4439947
0.4521946 	 0.440255
0.4574282 	 0.4363996
0.4619849 	 0.4327093
0.4668913 	 0.4288063
0.4715338 	 0.4253368
0.4764965 	 0.421645
0.4804857 	 0.4184383
0.4852011 	 0.4146869
0.4899177 	 0.4109764
0.4943524 	 0.4076133
0.4983545 	 0.4044911
0.5027009 	 0.4010284
0.5068534 	 0.3978851
0.5109861 	 0.3946461
0.5151621 	 0.3914088
0.5189329 	 0.3884144
0.5228269 	 0.3854301
0.526875 	 0.3820367
0.53058 	 0.3794101
0.534296 	 0.3763332
0.5380062 	 0.373504
0.5416169 	 0.3707832
0.5450933 	 0.3679549
0.5486781 	 0.3651633
0.5522888 	 0.3622587
0.555831 	 0.359488
0.5591854 	 0.356909
0.562613 	 0.3540276
0.5656977 	 0.3516996
0.5688442 	 0.3492151
0.5720281 	 0.3466708
0.5751904 	 0.3441564
0.5783597 	 0.341717
0.5813366 	 0.3392795
0.584296 	 0.3368841
0.58754 	 0.3344812
0.5902539 	 0.3322574
0.59303

## Experiment - exp theta

In [ ]:
const_theta = False
num_of_customers = 1_000_000_0

for lamba in np.arange(0.1,20.1,0.1):
    lamb = lamba
    arrival_time = 0
    pre_cus_time = 0
    cus = make_customer(pre_cus_time, id=0)

    queue = []

    num_blocked_users = 0
    num_deadlined_users = 0

    events = []

    time = 0
    cnt = 0

    events.append(Event(cnt, type_idx=0, occ_time=cus.arrival_time))
    pre_cus = cus
    cnt += 1

    nxt_cus = make_customer(cus.arrival_time, 1)
    done = False

    # main loop
    while len(events) > 0 or cnt < num_of_customers:
        if cnt < num_of_customers:
            if len(events) == 0:
                events.append(Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
                cnt += 1
                pre_cus = nxt_cus
                nxt_cus = make_customer(pre_cus.arrival_time, cnt)
            elif nxt_cus.arrival_time < events[0].occ_time:
                events.insert(0, Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
                cnt += 1
                pre_cus = nxt_cus
                nxt_cus = make_customer(pre_cus.arrival_time, cnt)

        event = events.pop(0)
        time = event.occ_time

        # arrival customer event
        if event.type_idx == 0:
            if len(queue) >= 12:
                num_blocked_users += 1
            else:
                customer = pre_cus
                queue.append(customer)

                if len(queue) == 1:
                    insert_arr(events, Event(event.customer_id, 2, time + customer.service_time))
                else:
                    insert_arr(events, Event(event.customer_id, 1, time + customer.waiting_time))

        # dead line event
        if event.type_idx == 1:
            customer = next((x for x in queue if x.id == event.customer_id), None)

            if customer and (customer.id != queue[0].id):
                queue.remove(customer)
                num_deadlined_users += 1

        # done event
        if event.type_idx == 2:
            customer = next((x for x in queue if x.id == event.customer_id), None)
            queue.remove(customer)

            # if exist a customer in queue
            if len(queue) >= 1:
                cus = queue[0]

                # remove deadline event
                for event in events:
                    if event.customer_id == cus.id and event.type_idx == 1:
                        events.remove(event)
                        break

                # add done event
                insert_arr(events, Event(cus.id, 2, time + cus.service_time))


    pb = num_blocked_users / num_of_customers
    pd = num_deadlined_users / num_of_customers

    print(pb, "\t", pd)

0.0 	 0.0333105
0.0 	 0.0665484
0.0 	 0.099322
0.0 	 0.1318565
0.0 	 0.1641416
0.0 	 0.1955413
0.0 	 0.2261256
0.0 	 0.2558051
0.0 	 0.2849284
4e-07 	 0.3126486
4e-07 	 0.339662
1.9e-06 	 0.3660382
3.1e-06 	 0.3912159
5.4e-06 	 0.4151231
1.22e-05 	 0.4378987
2.25e-05 	 0.4597046
4.69e-05 	 0.4806
7.74e-05 	 0.500616
0.0001281 	 0.5193276
0.0002026 	 0.5371378
0.0003054 	 0.5537027
0.0004763 	 0.5701797
0.000666 	 0.5847488
0.0009662 	 0.5989309
0.001329 	 0.6121054
0.0018199 	 0.624425
0.0024256 	 0.6363761
0.0031887 	 0.6470794
0.0041236 	 0.6572404
0.0051689 	 0.6662192
0.0065736 	 0.6750692
0.008164 	 0.6823982
0.0099695 	 0.6898984
0.0119697 	 0.6963615
0.0144921 	 0.7019234
0.017048 	 0.7067228
0.0200374 	 0.711
0.0233342 	 0.7143913
0.0267615 	 0.7177351
0.0308051 	 0.7201049
0.0347266 	 0.7219502
0.039242 	 0.722858
0.043942 	 0.7240198
0.0491437 	 0.724058
0.0543507 	 0.7237117
0.0602705 	 0.7225997
0.0657451 	 0.7216874
0.0716813 	 0.7202091
0.0781236 	 0.7179368
0.0841937 	 0

In [ ]:
const_theta = False
num_of_customers = 1_000_000_0

for lamba in np.arange(15.2,20.1,0.1):
    lamb = lamba
    arrival_time = 0
    pre_cus_time = 0
    cus = make_customer(pre_cus_time, id=0)

    queue = []

    num_blocked_users = 0
    num_deadlined_users = 0

    events = []

    time = 0
    cnt = 0

    events.append(Event(cnt, type_idx=0, occ_time=cus.arrival_time))
    pre_cus = cus
    cnt += 1

    nxt_cus = make_customer(cus.arrival_time, 1)
    done = False

    # main loop
    while len(events) > 0 or cnt < num_of_customers:
        if cnt < num_of_customers:
            if len(events) == 0:
                events.append(Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
                cnt += 1
                pre_cus = nxt_cus
                nxt_cus = make_customer(pre_cus.arrival_time, cnt)
            elif nxt_cus.arrival_time < events[0].occ_time:
                events.insert(0, Event(cnt, type_idx=0, occ_time=nxt_cus.arrival_time))
                cnt += 1
                pre_cus = nxt_cus
                nxt_cus = make_customer(pre_cus.arrival_time, cnt)

        event = events.pop(0)
        time = event.occ_time

        # arrival customer event
        if event.type_idx == 0:
            if len(queue) >= 12:
                num_blocked_users += 1
            else:
                customer = pre_cus
                queue.append(customer)

                if len(queue) == 1:
                    insert_arr(events, Event(event.customer_id, 2, time + customer.service_time))
                else:
                    insert_arr(events, Event(event.customer_id, 1, time + customer.waiting_time))

        # dead line event
        if event.type_idx == 1:
            customer = next((x for x in queue if x.id == event.customer_id), None)

            if customer and (customer.id != queue[0].id):
                queue.remove(customer)
                num_deadlined_users += 1

        # done event
        if event.type_idx == 2:
            customer = next((x for x in queue if x.id == event.customer_id), None)
            queue.remove(customer)

            # if exist a customer in queue
            if len(queue) >= 1:
                cus = queue[0]

                # remove deadline event
                for event in events:
                    if event.customer_id == cus.id and event.type_idx == 1:
                        events.remove(event)
                        break

                # add done event
                insert_arr(events, Event(cus.id, 2, time + cus.service_time))


    pb = num_blocked_users / num_of_customers
    pd = num_deadlined_users / num_of_customers

    print(pb, "\t", pd)

0.59326 	 0.3409786
0.5957554 	 0.3388024
0.5982205 	 0.3368411
0.6007291 	 0.3348019
0.6028203 	 0.3330454
0.605492 	 0.3309408
0.6075841 	 0.3290642
0.6097651 	 0.3272903
0.6122149 	 0.3253296
0.6144772 	 0.3233787
0.616798 	 0.3214266
0.6193045 	 0.3193626
0.6210884 	 0.3179453
0.6230248 	 0.3162878
0.6250341 	 0.3145744
0.6274052 	 0.3126727
0.6296659 	 0.3107371
0.6313281 	 0.3094817
0.6336151 	 0.3075181
0.6360936 	 0.3054416
0.6375414 	 0.3042807
0.640164 	 0.3021564
0.6412665 	 0.3012715
0.6433021 	 0.2994932
0.6453053 	 0.2979611
0.6473222 	 0.2962321
0.6490939 	 0.2946575
0.6511562 	 0.2929987
0.6528848 	 0.2915626
0.655036 	 0.2898583
0.6566548 	 0.2885136
0.6585239 	 0.286964
0.659719 	 0.2858717
0.6619562 	 0.2840116
0.6640619 	 0.2821076
0.6652342 	 0.2813376
0.6666382 	 0.2801482
0.6688376 	 0.2783913
0.6702248 	 0.2771053
0.6717953 	 0.2757905
0.6735936 	 0.274399
0.6752288 	 0.2729633
0.6769279 	 0.271514
0.6782439 	 0.2704358
0.6797436 	 0.2694131
0.681416 	 0.267832
